# Winners Analysis - Council of Alphas

Portfolio simulation: $100k initial capital, 1% risk per trade, ATR-based position sizing (TBM 3.0/1.5/24, 1h candles).

1. **Consensus Gate** (#1)
2. **Regime Router** (#2)
3. **Weighted Combination** (#3)

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

INITIAL_CAPITAL = 100_000

# Load trade logs (ranked by fitness: consensus > router > weighted)
consensus = pd.read_csv('../data/results/ranked/1_consensus_gate/trade_log.csv', parse_dates=['entry_ts', 'exit_ts'])
router = pd.read_csv('../data/results/ranked/2_regime_router/trade_log.csv', parse_dates=['entry_ts', 'exit_ts'])
weighted = pd.read_csv('../data/results/ranked/3_weighted_combination/trade_log.csv', parse_dates=['entry_ts', 'exit_ts'])

# Map each trade to its TBM outcome from state matrix
state = pd.read_parquet('../data/state_matrix_1h.parquet')
long_outcomes = state['tbm_long_outcome'].astype(str).to_numpy(copy=False)
short_outcomes = state['tbm_short_outcome'].astype(str).to_numpy(copy=False)

def attach_tbm_outcome(df):
    out = df.copy()
    entries = out['entry_index'].to_numpy(dtype=np.int64)
    sides = out['side'].to_numpy(dtype=np.int8)
    outcomes = np.full(len(out), 'TIMEOUT', dtype=object)
    long_mask = sides == 1
    short_mask = sides == -1
    outcomes[long_mask] = long_outcomes[entries[long_mask]]
    outcomes[short_mask] = short_outcomes[entries[short_mask]]
    out['tbm_outcome'] = outcomes
    return out

strategies = {
    'Consensus Gate (#1)': attach_tbm_outcome(consensus),
    'Regime Router (#2)': attach_tbm_outcome(router),
    'Weighted Combination (#3)': attach_tbm_outcome(weighted),
}

colors = {
    'Consensus Gate (#1)': '#EF553B',
    'Regime Router (#2)': '#00CC96',
    'Weighted Combination (#3)': '#636EFA',
}

## 1. Head - First Trades

In [2]:
for name, df in strategies.items():
    print(f'=== {name} ({len(df):,} trades) ===')
    display(df.head(10))
    print()

=== Consensus Gate (#1) (95 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime
0,2022-01-05 18:00:00+00:00,2022-01-05 19:00:00+00:00,114,115,1,-1,0.019504,101950.436492,NY,DOWNTREND,LOW_VOL
1,2022-01-11 22:00:00+00:00,2022-01-12 13:00:00+00:00,262,277,15,1,0.019637,103952.450270,OTHER,DOWNTREND,LOW_VOL
2,2022-02-13 19:00:00+00:00,2022-02-14 06:00:00+00:00,1051,1062,11,-1,-0.010342,102877.342786,NY,DOWNTREND,LOW_VOL
3,2022-02-18 20:00:00+00:00,2022-02-19 20:00:00+00:00,1172,1196,24,-1,0.001954,103078.384369,NY,DOWNTREND,LOW_VOL
4,2022-03-13 21:00:00+00:00,2022-03-13 22:00:00+00:00,1725,1726,1,-1,0.019448,105083.044248,OTHER,DOWNTREND,HIGH_VOL
5,2022-03-13 23:00:00+00:00,2022-03-14 04:00:00+00:00,1727,1732,5,-1,-0.010473,103982.549845,OTHER,DOWNTREND,HIGH_VOL
6,2022-04-30 19:00:00+00:00,2022-04-30 22:00:00+00:00,2875,2878,3,-1,0.019489,106009.100162,NY,DOWNTREND,LOW_VOL
7,2022-05-22 16:00:00+00:00,2022-05-23 07:00:00+00:00,3400,3415,15,1,0.019607,108087.587048,NY,CONSOLIDATION,HIGH_VOL
8,2022-06-01 16:00:00+00:00,2022-06-01 20:00:00+00:00,3640,3644,4,-1,0.019717,110218.718352,NY,DOWNTREND,LOW_VOL
9,2022-06-03 13:00:00+00:00,2022-06-04 02:00:00+00:00,3685,3698,13,-1,0.019726,112392.907521,NY,DOWNTREND,LOW_VOL



=== Regime Router (#2) (1,905 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime
0,2022-01-02 00:00:00+00:00,2022-01-02 04:00:00+00:00,24,28,4,1,-0.010555,98944.503497,ASIA,CONSOLIDATION,LOW_VOL
1,2022-01-02 19:00:00+00:00,2022-01-03 16:00:00+00:00,43,64,21,-1,0.019457,100869.715535,NY,CONSOLIDATION,HIGH_VOL
2,2022-01-03 21:00:00+00:00,2022-01-04 11:00:00+00:00,69,83,14,-1,-0.010541,99806.424363,OTHER,DOWNTREND,HIGH_VOL
3,2022-01-04 21:00:00+00:00,2022-01-05 17:00:00+00:00,93,113,20,-1,0.019558,101758.431887,OTHER,DOWNTREND,HIGH_VOL
4,2022-01-05 18:00:00+00:00,2022-01-05 19:00:00+00:00,114,115,1,-1,0.019504,103743.165476,NY,DOWNTREND,LOW_VOL
5,2022-01-05 20:00:00+00:00,2022-01-05 22:00:00+00:00,116,118,2,-1,0.019603,105776.873819,NY,DOWNTREND,HIGH_VOL
6,2022-01-05 23:00:00+00:00,2022-01-06 10:00:00+00:00,119,130,11,-1,0.019706,107861.362426,OTHER,DOWNTREND,HIGH_VOL
7,2022-01-06 11:00:00+00:00,2022-01-06 17:00:00+00:00,131,137,6,-1,-0.010232,106757.739993,LONDON,DOWNTREND,HIGH_VOL
8,2022-01-06 22:00:00+00:00,2022-01-07 00:00:00+00:00,142,144,2,1,-0.010307,105657.357118,OTHER,DOWNTREND,LOW_VOL
9,2022-01-07 02:00:00+00:00,2022-01-07 04:00:00+00:00,146,148,2,-1,0.019706,107739.440461,ASIA,DOWNTREND,LOW_VOL



=== Weighted Combination (#3) (2,739 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime
0,2022-01-01 23:00:00+00:00,2022-01-02 04:00:00+00:00,23,28,5,1,-0.010545,98945.505453,OTHER,CONSOLIDATION,LOW_VOL
1,2022-01-02 19:00:00+00:00,2022-01-03 16:00:00+00:00,43,64,21,-1,0.019457,100870.736987,NY,CONSOLIDATION,HIGH_VOL
2,2022-01-03 17:00:00+00:00,2022-01-04 14:00:00+00:00,65,86,21,-1,-0.010554,99806.151159,NY,DOWNTREND,HIGH_VOL
3,2022-01-04 15:00:00+00:00,2022-01-04 18:00:00+00:00,87,90,3,1,-0.010456,98762.589668,NY,DOWNTREND,HIGH_VOL
4,2022-01-04 20:00:00+00:00,2022-01-05 18:00:00+00:00,92,114,22,-1,0.019567,100695.104579,NY,DOWNTREND,HIGH_VOL
5,2022-01-05 19:00:00+00:00,2022-01-05 22:00:00+00:00,115,118,3,-1,0.019568,102665.477302,NY,DOWNTREND,HIGH_VOL
6,2022-01-05 23:00:00+00:00,2022-01-06 10:00:00+00:00,119,130,11,-1,0.019706,104688.651272,OTHER,DOWNTREND,HIGH_VOL
7,2022-01-06 11:00:00+00:00,2022-01-06 17:00:00+00:00,131,137,6,-1,-0.010232,103617.491577,LONDON,DOWNTREND,HIGH_VOL
8,2022-01-06 18:00:00+00:00,2022-01-07 00:00:00+00:00,138,144,6,1,-0.010236,102556.910392,NY,DOWNTREND,HIGH_VOL
9,2022-01-07 02:00:00+00:00,2022-01-07 04:00:00+00:00,146,148,2,-1,0.019706,104577.896348,ASIA,DOWNTREND,LOW_VOL


## 2. Info - DataFrame Structure

In [3]:
for name, df in strategies.items():
    print(f'=== {name} ===')
    print(f'Shape: {df.shape}')
    print()
    df.info()
    print('\n')

=== Consensus Gate (#1) ===
Shape: (95, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   entry_ts          95 non-null     datetime64[ns, UTC]
 1   exit_ts           95 non-null     datetime64[ns, UTC]
 2   entry_index       95 non-null     int64              
 3   exit_index        95 non-null     int64              
 4   duration          95 non-null     int64              
 5   side              95 non-null     int64              
 6   net_trade_return  95 non-null     float64            
 7   account_balance   95 non-null     float64            
 8   session           95 non-null     object             
 9   trend_regime      95 non-null     object             
 10  vol_regime        95 non-null     object             
dtypes: datetime64[ns, UTC](2), float64(2), int64(4), object(3)
memory usage: 8.3+ KB

## 3. Portfolio Performance

In [4]:
# net_trade_return is now the LEVERAGED portfolio return (not raw asset return)
# account_balance tracks compounding equity
for name, df in strategies.items():
    returns = df['net_trade_return']
    outcomes = df['tbm_outcome']
    final_equity = df['account_balance'].iloc[-1]
    total_return = (final_equity - INITIAL_CAPITAL) / INITIAL_CAPITAL
    
    # Compounded check (should match account_balance)
    comp_return = (1 + returns).prod() - 1
    
    # Win/loss based on TBM outcomes only (TP/SL). Timeouts excluded.
    tp_returns = returns[outcomes == 'TP']
    sl_returns = returns[outcomes == 'SL']
    timeout_count = int((outcomes == 'TIMEOUT').sum())
    wins = int(len(tp_returns))
    losses = int(len(sl_returns))
    scored = wins + losses
    wr = (wins / scored) if scored > 0 else np.nan
    avg_win = tp_returns.mean()
    avg_loss = sl_returns.mean()
    rr = avg_win / abs(avg_loss) if pd.notna(avg_loss) and avg_loss != 0 else np.nan
    
    # Drawdown
    peak = df['account_balance'].cummax()
    drawdown = (df['account_balance'] - peak) / peak
    max_dd = drawdown.min()
    
    print(f'=== {name} ===')
    print(f'  Total trades:       {len(df):,}')
    print(f'  Final equity:       ${final_equity:,.2f}')
    print(f'  Total return:       {total_return:+.2%}')
    print(f'  Max drawdown:       {max_dd:.2%}')
    print(f'  Win rate (TP/SL):   {wr:.1%}' if pd.notna(wr) else '  Win rate (TP/SL):   N/A')
    print(f'  TP hits / SL hits:  {wins:,} / {losses:,}')
    print(f'  Timeouts:           {timeout_count:,}')
    print(f'  Avg win (portfolio):{avg_win*100:>+10.4f}%')
    print(f'  Avg loss (portfolio):{avg_loss*100:>+9.4f}%')
    print(f'  Risk-reward:        {rr:.2f}')
    print()

=== Consensus Gate (#1) ===
  Total trades:       95
  Final equity:       $167,092.36
  Total return:       +67.09%
  Max drawdown:       -6.86%
  Win rate:           55.8%
  Wins / Losses:      53 / 42
  Avg win (portfolio):   +1.7598%
  Avg loss (portfolio):  -0.9723%
  Risk-reward:        1.81

=== Regime Router (#2) ===
  Total trades:       1,905
  Final equity:       $233,046.31
  Total return:       +133.05%
  Max drawdown:       -31.30%
  Win rate:           39.6%
  Wins / Losses:      755 / 1,150
  Avg win (portfolio):   +1.6529%
  Avg loss (portfolio):  -0.9964%
  Risk-reward:        1.66

=== Weighted Combination (#3) ===
  Total trades:       2,739
  Final equity:       $144,581.64
  Total return:       +44.58%
  Max drawdown:       -43.16%
  Win rate:           38.6%
  Wins / Losses:      1,056 / 1,683
  Avg win (portfolio):   +1.6501%
  Avg loss (portfolio):  -0.9987%
  Risk-reward:        1.65



## 4. Sharpe Ratio Analysis

In [5]:
# Load state matrix to compute per-trade leverage (needed for zero-fee estimate)
state = pd.read_parquet('../data/state_matrix_1h.parquet')
close_arr = state['close'].values
atr_arr = state['ATR_24'].values

FEE = 0.00075
TBM_LOSS = 1.5
RISK_PCT = 0.01

for name, df in strategies.items():
    returns = df['net_trade_return']
    
    mean_ret = returns.mean()
    std_ret = returns.std()
    sharpe = mean_ret / std_ret * np.sqrt(len(df)) if std_ret > 0 else 0
    per_trade_sharpe = mean_ret / std_ret if std_ret > 0 else 0
    
    # Zero-fee estimate: add back leverage * fee per trade
    entries = df['entry_index'].values
    sl_pct = TBM_LOSS * atr_arr[entries] / close_arr[entries]
    leverage = np.minimum(RISK_PCT / sl_pct, 20.0)
    gross_returns = returns + leverage * FEE
    
    gross_mean = gross_returns.mean()
    gross_std = gross_returns.std()
    gross_sharpe = gross_mean / gross_std if gross_std > 0 else 0
    
    print(f'=== {name} ===')
    print(f'  Mean return per trade:  {mean_ret:.6f} ({mean_ret*100:.4f}%)')
    print(f'  Std dev of returns:     {std_ret:.6f} ({std_ret*100:.4f}%)')
    print(f'  Per-trade Sharpe:       {per_trade_sharpe:.4f}')
    print(f'  Sharpe (sqrt-N scaled): {sharpe:.4f}')
    print(f'  Zero-fee Sharpe:        {gross_sharpe:.4f}  (gross mean={gross_mean*100:.4f}%)')
    print(f'  Avg leverage:           {leverage.mean():.2f}x')
    print(f'  Min return:             {returns.min():.6f} ({returns.min()*100:.4f}%)')
    print(f'  Max return:             {returns.max():.6f} ({returns.max()*100:.4f}%)')
    print(f'  Skewness:               {returns.skew():.4f}')
    print(f'  Kurtosis:               {returns.kurtosis():.4f}')
    print()

=== Consensus Gate (#1) ===
  Mean return per trade:  0.005519 (0.5519%)
  Std dev of returns:     0.014288 (1.4288%)
  Per-trade Sharpe:       0.3863
  Sharpe (sqrt-N scaled): 3.7649
  Zero-fee Sharpe:        0.4150  (gross mean=0.5928%)
  Avg leverage:           0.54x
  Min return:             -0.011220 (-1.1220%)
  Max return:             0.019923 (1.9923%)
  Skewness:               -0.1066
  Kurtosis:               -1.9299

=== Regime Router (#2) ===
  Mean return per trade:  0.000536 (0.0536%)
  Std dev of returns:     0.013568 (1.3568%)
  Per-trade Sharpe:       0.0395
  Sharpe (sqrt-N scaled): 1.7233
  Zero-fee Sharpe:        0.0701  (gross mean=0.0952%)
  Avg leverage:           0.56x
  Min return:             -0.011618 (-1.1618%)
  Max return:             0.019945 (1.9945%)
  Skewness:               0.5650
  Kurtosis:               -1.5554

=== Weighted Combination (#3) ===
  Mean return per trade:  0.000225 (0.0225%)
  Std dev of returns:     0.013482 (1.3482%)
  Per-trade Sh

## 5. Dollar Equity Curve ($100k start, 1% risk)

In [6]:
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=[
        'Consensus Gate (#1)',
        'Regime Router (#2)',
        'Weighted Combination (#3)',
    ],
    vertical_spacing=0.08,
)

for i, (name, df) in enumerate(strategies.items(), 1):
    fig.add_trace(
        go.Scatter(
            x=df['entry_ts'],
            y=df['account_balance'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=1.5),
        ),
        row=i, col=1,
    )
    
    fig.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', opacity=0.5, row=i, col=1)
    fig.update_yaxes(title_text='Equity ($)', tickformat='$,.0f', row=i, col=1)

fig.update_xaxes(title_text='Date', row=3, col=1)

fig.update_layout(
    height=1000,
    title_text='Dollar Equity Curve - $100k, 1% Risk Per Trade (TBM 3.0/1.5/24, 1h)',
    showlegend=True,
    template='plotly_white',
)

fig.show()

In [7]:
# Overlay comparison
fig2 = go.Figure()

for name, df in strategies.items():
    fig2.add_trace(
        go.Scatter(
            x=df['entry_ts'],
            y=df['account_balance'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=1.5),
        )
    )

fig2.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', opacity=0.5)

fig2.update_layout(
    height=500,
    title_text='Equity Overlay - All Three Hybrids ($100k Start)',
    yaxis_title='Equity ($)',
    yaxis_tickformat='$,.0f',
    xaxis_title='Date',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
)

fig2.show()

## 6. Average Holding Time

In [8]:
# duration column is in bars (1h candles)
for name, df in strategies.items():
    holding_hours = df['duration']  # 1 bar = 1 hour on 1h data
    
    print(f'=== {name} ===')
    print(f'  Mean:   {holding_hours.mean():.2f} hours')
    print(f'  Median: {holding_hours.median():.2f} hours')
    print(f'  Min:    {holding_hours.min()} hours')
    print(f'  Max:    {holding_hours.max()} hours')
    print(f'  Hit time barrier (duration=24 bars / 24h): {(df["duration"] == 24).sum():,} ({(df["duration"] == 24).mean():.1%})')
    print()

=== Consensus Gate (#1) ===
  Mean:   11.14 hours
  Median: 10.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 11 (11.6%)

=== Regime Router (#2) ===
  Mean:   10.39 hours
  Median: 8.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 256 (13.4%)

=== Weighted Combination (#3) ===
  Mean:   10.19 hours
  Median: 8.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 374 (13.7%)

